# Primary Current Distribution Demo

This demo uses Schwarz-Christoffel mapping to show lines of constant current and potential for two electrodes placed on the edge of a 2D rectangular domain.  The calculation is based on the following two references:
 1. V. I. Ivanov and M. K. Trubetskov, Handbook of Conformal Mapping With Computer-Aided Visualization, CRC Press, (1994).  
 2. R. Schinzinger and P. A. A. Laura, Conformal Mapping: Methods and Applications, Courier Corporation, (2012).  

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as pl
import mpmath as mp

from scipy.optimize import fsolve
import matplotlib.patches as patches
from ipywidgets import widgets, interact

In [2]:
sn = np.vectorize(mp.ellipfun('sn'))


def ellipj(t, m):
    z = mp.ellipf(mp.asin(t),m)
    z = np.real(z) + np.abs(np.imag(z))*1j
    return z


ellipjV = np.vectorize(ellipj)


mp_to_complex = np.vectorize(np.cdouble)

In [3]:
def primary_rect(AR=3.0,wr=0.2,lines=15):
    
    b, h= 0.5, AR
    mod=h/b
    qz=exp(-pi*mod)
    mz=mp.mfrom(q=qz)
    kz=sqrt(mz)

    Kz=mp.ellipk(mz)
    Bz=b/Kz

    # boundary coordinates
    pz = np.array([-b+h*1j, -b, b, b+h*1j])

    # electrode coordinates
    vz = np.array([ b + h/2*(1 - wr) * 1j,  b + h / 2 * (1 + wr) * 1j,
                -b + h/2*(1 + wr) * 1j, -b + h / 2 * (1 - wr) * 1j])

    # convert to t-space
    ptz = sn(pz/Bz,mz)
    vtz = sn(vz/Bz,mz)

    # convert to w-space
    at, bt, ct, dt = vtz
    Dv = (ct - at) / (ct - bt) / ((dt - at) / (dt - bt))
    mw = (mp.sqrt(Dv) - mp.sqrt(Dv - 1))**4
    kw = sqrt(real(mw))
    qw = mp.qfrom(m=mw)
    Bw = -mp.log(qw)/pi

    resistance= 2.0/float(real(Bw))  # = 2*Kw/Kpw

    tw = np.array([-1/kw, -1, 1, 1/kw])
    wtnew = ellipjV(tw,mw)
    wt=mp_to_complex(wtnew)
    
    # Linear fractional transformation
    t0 = sort(real(mp_to_complex(vtz)))
    t1 = sort(real(mp_to_complex(tw)))

    delta = 1.0
    ttrans= lambda a,t: (a[0]*t + a[1]) / (a[2]*t + delta)
    zerofun= lambda a: t1[1:] - ttrans(a,t0[1:])

    alpha, beta, gamma=fsolve(zerofun,[1,1,0],xtol=1.0e-12)
    
    # w-space grid
    Kw = real(np.cdouble(mp.ellipk(mw)))
    Kpw = real(np.cdouble(mp.ellipk(1 - mw)))

    x=linspace(-Kw, Kw, lines)
    y=linspace(0, Kpw, 101)
    phix, phiy = np.meshgrid(x, y)
    phiw= phix + phiy*1j

    x=linspace(-Kw, Kw, 101)
    y=linspace(0, Kpw, lines)
    dphix, dphiy = np.meshgrid(x, y)
    dphiw= dphix.T + dphiy.T*1j

    # Transform grid from w-space to z-space
    def w_to_z(w):
        
        tw = sn(w, mw)
        tz = (delta*tw - beta) / (-gamma*tw + alpha)
        z = Bz*ellipjV(tz,mz)

        return mp_to_complex(z)    
    
    phiz= w_to_z(phiw)
    dphiz= w_to_z(dphiw)
    
    # Plot
    compT= lambda z: (imag(z), real(z))

    rect = patches.Rectangle((0,-b),h,2*b,linewidth=1,edgecolor='k',facecolor='none')

    plot(*compT(vz[:2]),'r',linewidth=2)
    plot(*compT(vz[2:]),'b',linewidth=2)
    plot(*compT(phiz), color='gray', linewidth=0.5);
    plot(*compT(dphiz), color='gray', linewidth=0.5, linestyle='dashed')
    gca().add_patch(rect)

    axis('equal')
    axis('off')

    rstring='$R/R_0 = $ {0:5.3f}'.format(resistance)
    text( 0.5, 1.05, rstring,fontsize=10)

    show()

In [4]:
interact(primary_rect, AR=[1/3, 1/2, 1, 2, 3], wr=[0.1, 0.2, 1/3, 0.5, 1], lines=[5,10,15,20]);

interactive(children=(Dropdown(description='AR', index=4, options=(0.3333333333333333, 0.5, 1, 2, 3), value=3)…